In [1]:
// id : ex_4.1
sealed trait Option[+A] {
    def map[B](f: A => B) : Option[B] = {
        this match {
            case Some(a) => Some(f(a))
            case None => None
        }
    }
    def flatMap[B](f: A => Option[B]) : Option[B] = {
        this match {
            case Some(a) => f(a)
            case None => None
        }
    }
    def getOrElse[B >: A](default: => B) : B = {
        this match {
            case Some(a) => a
            case None => default
        }
    }
    def orElse[B >: A](ob: => Option[B]) : Option[B] = {
        this match {
            case Some(a) => Some(a)
            case None => ob
        }
    }
    def filter(f: A => Boolean) : Option[A] = {
        this match {
            case Some(a) => if( f(a) ) { this } else { None }
            case None => None
        }
    }
}
case class Some[+A](get: A) extends Option[A]
case object None extends Option[Nothing]

None

In [5]:
// id : try ex_4.1
println(Some(3).map(_ + 1))
val NoneInt : Option[Int] = None
println(NoneInt.map(_ + 1))

Some(4)
None


null